# Détection des mélanomes par apprentissage machine

## Introduction
Ce notebook présente le développement d’un modèle d’apprentissage machine pour classifier les lésions cutanées en `bénignes` ou `maligne`. Le dataset utilisé provient de l’ISIC (International Skin Imaging Collaboration). Le projet s’appuie sur le transfer learning avec le modèle EfficientNet. Les étapes incluent le prétraitement des données, l’implémentation du modèle, l’entraînement et l’évaluation.

## Importation des bibliothèques
Nous commençons par installer et importer les bibliothèques nécessaires pour notre projet. Ces bibliothèques permettent de manipuler les données, d’afficher des visualisations et de créer le modèle d’apprentissage. Les principales bibliothèques utilisées sont :
- `numpy` et `pandas` pour la manipulation des données,
- `tensorflow.keras` pour la construction du modèle,
- d’autres modules spécifiques pour le prétraitement et la visualisation.

In [2]:
%pip install tensorflow
%pip install matplotlib
%pip install numpy
%pip install pandas
%pip install pydot
%pip install pydot graphviz
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.



In [25]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras.applications import EfficientNetB0 # type: ignore #ingore the warning
from tensorflow.keras.layers import Concatenate, Input, Dense, Dropout, Flatten, Conv2D, MaxPooling2D,GlobalAveragePooling2D # type: ignore #ingore the warning
from tensorflow.keras.models import Model, Sequential # type: ignore #ingore the warning
from tensorflow.keras.optimizers import Adam # type: ignore #ingore the warning
from tensorflow.keras.utils import plot_model # type: ignore #ingore the warning
from tensorflow.keras.preprocessing.image import load_img, img_to_array # type: ignore #ingore the warning
from tensorflow.keras.applications.efficientnet import preprocess_input # type: ignore #ingore the warning
from tensorflow.keras.callbacks import TensorBoard # type: ignore #ingore the warning
from tensorflow.keras.metrics import Precision, Recall, Metric # type: ignore #ingore the warning
from keras.saving import register_keras_serializable # type: ignore #ingore the warning
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model

## Détection du matériel (TPU, GPU, CPU)

Cette cellule détecte le matériel disponible (TPU, GPU ou CPU) et configure une stratégie adaptée pour optimiser l'entraînement :

1. **Priorité** : TPU > GPU > CPU.
2. **Stratégie de distribution des calculs** :
   - TPU : Utilisation de `TPUStrategy` pour exécutions distribuées.
   - GPU : Utilisation de `MirroredStrategy` pour un ou plusieurs GPU.
   - CPU : Stratégie par défaut si pas de TPU ou GPU disponible.
3. **Ressources synchronisées** : Le nombre de répliques (`strategy.num_replicas_in_sync`) sera utilisé pour ajuster dynamiquement la taille globale du batch.

Cette configuration optimise l'utilisation des ressources matérielles (en puissance de calcul) disponibles.

In [4]:
# Detect hardware
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
except ValueError:
  tpu = None
#If TPU not found try with GPUs
  gpus = tf.config.experimental.list_logical_devices("GPU")
    
# Select appropriate distribution strategy for hardware
if tpu:
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu)
  print('Running on TPU ', tpu.master())  
elif len(gpus) > 0:
  strategy = tf.distribute.MirroredStrategy(gpus) # this works for 1 to multiple GPUs
  print('Running on ', len(gpus), ' GPU(s) ')
else:
  strategy = tf.distribute.get_strategy()
  print('Running on CPU')

# How many accelerators do we have ?
print("Number of accelerators: ", strategy.num_replicas_in_sync)

Running on CPU
Number of accelerators:  1


TFRECORDS

In [5]:
def _bytes_feature(value):
    """Retourne une fonction de type tf.train.Feature pour une valeur binaire (image)."""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[tf.io.encode_jpeg(value).numpy()]))

def _int64_feature(value):
    """Retourne une fonction de type tf.train.Feature pour une valeur entière (label)."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def image_example(image, label):
    """Convertit une image et un label en un format tf.train.Example."""
    feature = {
        'image': _bytes_feature(image),
        'label': _int64_feature(label)
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

def write_tfrecords(metadata_df, output_path):

    if not os.path.exists('tfrecords'):
        os.makedirs('tfrecords')
    
    """Crée un fichier TFRecord à partir du DataFrame contenant les chemins des images et les labels."""
    with tf.io.TFRecordWriter(output_path) as writer:
        for _, row in metadata_df.iterrows():
            # Charger l'image à partir du chemin
            image_path = row['image_path']
            image = tf.io.read_file(image_path)
            image = tf.image.decode_jpeg(image, channels=3)  # Décode l'image au format JPEG
            
            # Récupérer l'étiquette
            label = row['label']
            
            # Convertir l'image et l'étiquette en un example TFRecord
            tf_example = image_example(image, label)
            
            # Écrire l'exemple dans le fichier TFRecord
            writer.write(tf_example)

Définition des paramètres globaux pour l'entrainement du modèle

In [6]:
# Chemins vers les fichiers TFRecord (désactivé dans cet exemple)
# TRAINING_FILENAMES = tf.io.gfile.glob(GCS_DS_PATH + '/tfrecords/train*')
# TEST_FILENAMES = tf.io.gfile.glob(GCS_DS_PATH + '/tfrecords/test*')

# Taille des lots adaptée au nombre de répliques (dispositifs GPU/TPU) disponibles

##################################
# rajout conidtion si GPU ou TPU #
##################################
BATCH_SIZE = 32 * strategy.num_replicas_in_sync  # Par exemple, 10 images par réplique

# Taille des images à utiliser (100x100 pixels)
IMAGE_SIZE = [100, 100]  # Taille utilisée pour le redimensionnement des images
imSize = 100             # Taille utilisée pour redimensionner les images dans les pipelines

# Optimisation automatique pour le préchargement des données
AUTO = tf.data.experimental.AUTOTUNE

# Nombre d'époques (itérations sur l'ensemble d'entraînement)
EPOCHS = 40  # Peut être ajusté en fonction des performances et de la convergence

# Définir l'entrée du modèle (couche d'entrée pour TensorFlow/Keras)
input_layer = Input(shape=(imSize, imSize, 3))  # Entrée avec une image RGB (3 canaux)

## Chargement des données
Les données, composées d’images et de métadonnées, sont chargées depuis un fichier CSV contenant les informations associées. Ces métadonnées incluent notamment :
- le label `benign_malignant` (notre variable cible),
- des informations sur le patient et la lésion.

L’affichage des premières lignes du fichier nous permet de vérifier sa structure et son contenu avant toute manipulation.

In [7]:
# Chargement du fichier CSV contenant les métadonnées
metadata = pd.read_csv("./Data/benign/cutout/metadata.csv",encoding='ISO-8859-1', sep=';')
metadata_norm = pd.read_csv("./Data/benign/metadata.csv",encoding='ISO-8859-1', sep=';')
metadata_clinique = pd.read_csv("./Data/train.csv")
# Ajouter les chemins complets des images

metadata['image_path'] = metadata['isic_id'].apply(lambda x: f"./Data/benign/cutout/{x}.jpg")
metadata['label'] = metadata['benign_malignant'].map({'benign': 0, 'malignant': 1})

metadata_clinique['image_path'] = metadata_clinique['image_name'].apply(lambda x: f"./Data/train/{x}.jpg")
metadata_clinique['label'] = metadata_clinique['benign_malignant'].map({'benign': 0, 'malignant': 1})

metadata_norm['image_path'] = metadata_norm['isic_id'].apply(lambda x: f"./Data/benign/{x}.jpg")
metadata_norm['label'] = metadata_norm['benign_malignant'].map({'benign': 0, 'malignant': 1})


# Supprimer les lignes avec des valeurs manquantes
metadata = metadata.dropna(subset=['label'])
metadata_norm = metadata_norm.dropna(subset=['label'])
metadata_clinique = metadata_clinique.dropna(subset=['label'])

metadata_clinique = metadata_clinique.sample(n=10000,random_state=42)

# Utiliser 50% des données normales pour l'entraînement
metadata_train = metadata_norm.sample(frac=0.5, random_state=42)

# Utiliser les 50% restants de metadata_norm pour le test
metadata_test2 = metadata_norm.drop(metadata_train.index)

# Concatenation de metadata (cutout) et metadata_train (50% des données normales) pour l'entraînement
metadata = pd.concat([metadata, metadata_train], ignore_index=True)

metadata = metadata [['image_path', 'label']]

metadata_clinique = metadata_clinique [['image_path', 'label']]

print(f"Nombre total d'images disponibles : {len(metadata)}")
print(f"Total rows in original CSV: {metadata.shape[0]}")

print(f"Nombre total d'images disponibles : {len(metadata_norm)}")

print(f"Nombre total d'images disponibles : {len(metadata_clinique)}")

Nombre total d'images disponibles : 5991
Total rows in original CSV: 5991
Nombre total d'images disponibles : 3994
Nombre total d'images disponibles : 10000


In [8]:
# Affiche le nombre  d'images par classe
print(metadata['label'].value_counts())

# Prendre un échantillon aléatoire de 10 000 images
# metadata = metadata.sample(n=5000, random_state=42)
print(metadata['label'].value_counts())
# Afficher la taille du nouvel échantillon
print(f"Nombre d'images après échantillonnage : {len(metadata)}")

label
0    4457
1    1534
Name: count, dtype: int64
label
0    4457
1    1534
Name: count, dtype: int64
Nombre d'images après échantillonnage : 5991


Division des données en ensembles d’entraînement, validation et test
À cette étape, les données sont divisées en trois ensembles distincts : entraînement, validation, et test, afin de préparer le modèle pour son entraînement et son évaluation. Tout d'abord, 70 % des données sont affectées à l'ensemble d'entraînement, qui sera utilisé pour ajuster les paramètres du modèle. Les 30 % restants sont répartis entre les ensembles de validation (20 % des données totales) et de test (10 % des données totales). La stratification est appliquée sur la variable label pour garantir que les proportions des classes (par exemple, "Malignant" et "Benign") sont similaires dans les trois ensembles. Cette méthode assure une représentation équilibrée des classes dans chaque ensemble, ce qui est essentiel pour éviter les biais dans l'évaluation des performances du modèle.

La stratification consiste à répartir les données de manière à ce que la proportion de chaque classe soit conservée dans les sous-ensembles créés (entraînement, validation, test). Cela garantit que les ensembles reflètent fidèlement la distribution initiale des classes, évitant les déséquilibres qui pourraient biaiser l'entraînement ou l'évaluation du modèle.

In [9]:
# Mélanger les données initiales
metadata = metadata.sample(frac=1, random_state=42).reset_index(drop=True)

# Division des ensembles en entraînement (70%), validation (20%), et test (10%)
# training_metadata = metadata.sample(frac=0.8, random_state=42)
# validation_metadata = metadata.drop(training_metadata.index)
train_size = int(0.7 * len(metadata))  # 70% pour l'entraînement
val_size = int(0.2 * len(metadata))    # 20% pour la validation
test_size = len(metadata) - train_size - val_size  # 10% pour le test

training_metadata = metadata.iloc[:train_size]
validation_metadata = metadata.iloc[train_size:train_size + val_size]
test_metadata = metadata.iloc[train_size + val_size:]

print(f"Entraînement : {training_metadata.shape[0]} exemples")
print(f"Validation : {validation_metadata.shape[0]} exemples")
print(f"Test : {test_metadata.shape[0]} exemples")

####################################################################################
# ??? Ajouter la fonction de création des datasets pour TensorFlow juste après ??? #
####################################################################################

Entraînement : 4193 exemples
Validation : 1198 exemples
Test : 600 exemples


### Vérifications des datasets

Cette étape on vérifie la cohérence après la division des données :

1. **Distribution des classes** : Vérification que les proportions de classes (`benign`/`malignant`) sont respectées dans chaque ensemble.
2. **Taille totale** : Validation que la somme des échantillons des trois ensembles correspond au dataset initial.
3. **Absence de chevauchements** : Contrôle qu’aucun échantillon ne se trouve dans plusieurs ensembles, évitant les problèmes de fuite de données (`data leakage`).

Toute incohérence déclenche une erreur pour correction.

In [10]:
print("Class distribution in training set:")
print(training_metadata['label'].value_counts(normalize=True))
print("Class distribution in validation set:")
print(validation_metadata['label'].value_counts(normalize=True))
print("Class distribution in test set:")
print(test_metadata['label'].value_counts(normalize=True))

total_samples = training_metadata.shape[0] + validation_metadata.shape[0] + test_metadata.shape[0]
print(f"Total samples: {total_samples} (original: {metadata.shape[0]})")

# Check pour des overlaps entre les sets de données (data leakage), retourne une erreur si overlap
assert len(set(training_metadata.index) & set(validation_metadata.index)) == 0, "Overlap between train and validation sets!"
# assert len(set(val_metadata.index) & set(test_metadata.index)) == 0, "Overlap between validation and test sets!"

Class distribution in training set:
label
0    0.749583
1    0.250417
Name: proportion, dtype: float64
Class distribution in validation set:
label
0    0.736227
1    0.263773
Name: proportion, dtype: float64
Class distribution in test set:
label
0    0.72
1    0.28
Name: proportion, dtype: float64
Total samples: 5991 (original: 5991)


In [11]:
malign_count = len(test_metadata[test_metadata['label'] == 1])
# Afficher le résultat
print(f"Nombre de labels bénins : {malign_count}")

Nombre de labels bénins : 168


### Définition des fonctions pour charger et prétraiter les données
À cette étape, le code prépare un pipeline de traitement des images pour l'entraînement du modèle. Les images sont chargées à partir de leurs chemins, redimensionnées à une taille standard de 100x100 pixels, normalisées (valeurs entre 0 et 1), et associées à leurs étiquettes (labels). Ces transformations sont encapsulées dans une fonction `load_image_and_label`, appliquée via `tf.data.Dataset` pour créer un ensemble de données TensorFlow optimisé. Enfin, les données sont divisées en lots et préchargées pour accélérer l'entraînement.

In [12]:
# Fonction pour charger et prétraiter une image
def load_image_and_label(image_path, label):
    """
    Charge une image depuis son chemin, applique des prétraitements
    (normalisation, redimensionnement), et retourne l'image et son étiquette.
    """
    # Charger l'image depuis son chemin
    image = tf.io.read_file(image_path)  # Lire le fichier image
    image = tf.image.decode_jpeg(image, channels=3)  # Décode une image JPEG en RGB
    
    # Normaliser les valeurs des pixels (entre 0 et 1)
    image = tf.cast(image, tf.float32) / 255.0
    
    # Redimensionner l'image à une taille standard
    image = tf.image.resize(image, [imSize, imSize])

    label = tf.cast(label, tf.int64)
    
    return image, label

# Fonction pour convertir un DataFrame Pandas en dataset TensorFlow
def create_tf_dataset(metadata_df, batch_size):
    """
    Convertit un DataFrame contenant les chemins des images et les labels 
    en un dataset TensorFlow optimisé pour l'entraînement.
    """
    # Extraction des colonnes nécessaires depuis le DataFrame
    image_paths = metadata_df['image_path'].values
    labels = metadata_df['label'].values
    
    # Créer un dataset TensorFlow à partir des chemins et des étiquettes
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))

    dataset = dataset.map(
        lambda image_path, label: load_image_and_label(image_path, label),
        num_parallel_calls=tf.data.AUTOTUNE
    )
    
    # Appliquer la fonction de chargement et de prétraitement à chaque image du dataset
    # dataset = dataset.map(load_image_and_label, num_parallel_calls=AUTO)
    
    dataset = dataset.repeat()

    dataset = dataset.shuffle(1500)  # Mélange avec un tampon de 1500 éléments

    # Diviser en lots et activer le préchargement pour optimiser les performances
    dataset = dataset.batch(batch_size)
    dataset= dataset.prefetch(AUTO)

    
    return dataset

def create_tf_test_dataset(metadata_df, batch_size):
    """
    Convertit un DataFrame contenant les chemins des images et les labels 
    en un dataset TensorFlow optimisé pour l'entraînement.
    """
    # Extraction des colonnes nécessaires depuis le DataFrame
    image_paths = metadata_df['image_path'].values
    labels = metadata_df['label'].values
    dataset = tf.data.Dataset.from_tensor_slices(metadata_df.to_dict('list'))
    
    # Créer un dataset TensorFlow à partir des chemins et des étiquettes
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))

    dataset = dataset.map(
        lambda image_path, label: load_image_and_label(image_path, label),
        num_parallel_calls=tf.data.AUTOTUNE
    )

    # Diviser en lots et activer le préchargement pour optimiser les performances
    dataset = dataset.batch(batch_size)
    dataset= dataset.prefetch(AUTO)

    
    return dataset

Créations des TFrecords

In [13]:
def _parse_tfrecord(proto):
    """Fonction pour décoder un exemple à partir du fichier TFRecord."""
    # Description de l'exemple dans le fichier TFRecord
    description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.int64)
    }
    
    # Parser l'exemple
    parsed_features = tf.io.parse_single_example(proto, description)
    
    # Décoder l'image
    image = tf.io.decode_jpeg(parsed_features['image'], channels=3)
    # Redimensionner l'image
    image = tf.image.resize(image, [imSize, imSize])
    # Normaliser l'image
    image = tf.cast(image, tf.float32) / 255.0
    
    label = parsed_features['label']
    
    return image, label

def create_tf_dataset_from_tfrecord(tfrecord_files, batch_size):
    """Charge un dataset à partir des fichiers TFRecord."""
    # Créer un dataset à partir des fichiers TFRecord
    dataset = tf.data.TFRecordDataset(tfrecord_files)
    
    # Parser les éléments du TFRecord
    dataset = dataset.map(_parse_tfrecord, num_parallel_calls=tf.data.AUTOTUNE)
    
    # Appliquer les transformations (mélange, regroupement en lots, etc.)
    dataset = dataset.shuffle(1500).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    dataset = dataset.repeat()
    
    return dataset

#### Test d'affichage de quelques images du dataset (depuis le pipeline TensorFlow)
Cette étape permet de vérifier si les images ont été correctement redimensionnées en prenant aléatoirement 5 images.

In [14]:
# # Visualiser 5 images aléatoires depuis le pipeline TensorFlow
# for image, label in create_tf_dataset(metadata.sample(frac=1).reset_index(drop=True), batch_size=1).take(5):
#     # Convertir le tenseur en tableau NumPy pour l'affichage
#     img_resized = image[0].numpy()
#     plt.imshow(img_resized)
#     # Afficher le label avec le mapping 0 -> Benign, 1 -> Malignant
#     plt.title(f"{'Benign' if label.numpy() == 0 else 'Malignant'}")
#     plt.axis('off')  # Supprimer les axes pour une meilleure visualisation
#     plt.show()

Création des datasets TensorFlow

Cette étape transforme les sous-ensembles (entraînement, validation et test) en pipelines optimisés pour TensorFlow à l'aide de create_tf_dataset. Chaque dataset contient des images prétraitées (chargées, redimensionnées, et normalisées) associées à leurs labels, regroupées en lots de taille spécifiée (BATCH_SIZE). Ces datasets sont utilisés directement par le modèle lors de l'entraînement ou de l'évaluation pour assurer une gestion efficace des données.

In [15]:
# Créer les datasets
training_dataset = create_tf_dataset(training_metadata, batch_size=BATCH_SIZE)
validation_dataset = create_tf_dataset(validation_metadata, batch_size=BATCH_SIZE)
test_dataset = create_tf_test_dataset(test_metadata, batch_size=BATCH_SIZE)
test2_dataset = create_tf_test_dataset(metadata_test2, batch_size=BATCH_SIZE)
test3_dataset = create_tf_test_dataset(metadata_clinique, batch_size=BATCH_SIZE)



# Créer les fichiers TFRecord pour chaque partie de votre dataset
# write_tfrecords(training_metadata, 'tfrecords/train.tfrecords')
# write_tfrecords(validation_metadata, 'tfrecords/val.tfrecords')
# # write_tfrecords(test_metadata, 'test.tfrecords')

# # Utiliser les fichiers TFRecord pour créer les datasets
# training_dataset = create_tf_dataset_from_tfrecord(['tfrecords/train.tfrecords'], batch_size=BATCH_SIZE)
# validation_dataset = create_tf_dataset_from_tfrecord(['tfrecords/val.tfrecords'], batch_size=BATCH_SIZE)
# test_dataset = create_tf_dataset_from_tfrecord(['test.tfrecords.tfrec'], batch_size=BATCH_SIZE)

# print(f"Test dataset : {len(test_dataset)}")
# print(f"Test dataset : {len(test_dataset)} batches")

Calcul des paramètres pour l’entraînement
Les paramètres STEPS_PER_EPOCH et VALIDATION_STEPS déterminent le nombre de lots nécessaires pour parcourir une fois l'ensemble des données d'entraînement ou de validation. Ils sont calculés en divisant la taille totale des données par BATCH_SIZE pour garantir que le modèle traite toutes les données à chaque époque, tout en optimisant l'utilisation des ressources.

In [16]:
# Calcul des paramètres pour l'entraînement
# STEPS_PER_EPOCH = len(training_dataset) // BATCH_SIZE
# VALIDATION_STEPS = len(validation_dataset) // BATCH_SIZE 
steps_per_epochs = len(training_metadata) // BATCH_SIZE
validation_steps = len(validation_metadata) // BATCH_SIZE

print(f"Nombre d'étapes pour l'entraînement : {steps_per_epochs}")
print(f"Nombre d'étapes pour la validation : {validation_steps}")

Nombre d'étapes pour l'entraînement : 131
Nombre d'étapes pour la validation : 37


## Construction du modèle
### Chargement du modèle pré-entraîné
Nous utilisons `EfficientNetB0`, un modèle convolutionnel pré-entraîné sur ImageNet. Les couches internes, qui extraient les caractéristiques générales des images, sont gelées, tandis que la dernière couche de classification est remplacée pour s’adapter à notre tâche.

In [17]:
# Charger le modèle EfficientNetB0 préentraîné sans la couche de classification finale

base_model = EfficientNetB0(
            input_shape=(imSize, imSize, 3),
            weights='imagenet',
            include_top=False
        )
# Geler les couches du modèle de base pour conserver les poids préentraînés
base_model.trainable = True # On réalise du fine-tuning

In [18]:
# Passer l'entrée à travers le modèle de base
x = base_model(input_layer, training=True)  # `input_layer` 

# Ajouter des couches convolutionnelles et de pooling
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)

x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)


# Aplatir les caractéristiques extraites
x = GlobalAveragePooling2D()(x)

# Ajouter des couches fully connected avec dropout
x = Dense(1024, activation='relu')(x)
x = Dropout(0.4)(x)

x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)

x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)

x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)

x = Dense(64, activation='relu')(x)
x = Dropout(0.2)(x)

output = Dense(1, activation='sigmoid')(x)  # Sigmoid pour une classification binaire


# Définir le modèle final avec une seule entrée (image)
model = Model(inputs=input_layer, outputs=output)

### Compilation du modèle

Cette étape configure le modèle pour l’entraînement en définissant l’optimiseur, la fonction de perte et les métriques à suivre. Après la compilation, un résumé du modèle est affiché pour vérifier la structure des couches, le nombre de paramètres et la compatibilité des dimensions.

In [26]:
# Calcul des poids des classes pour l'entraînement


# benin_weight = len(training_metadata) / len(test_metadata[test_metadata['label'] == 0])
# malin_weight = len(training_metadata) / len(test_metadata[test_metadata['label'] == 1])

# print(f"Poids de la classe bénigne : {benin_weight:.2f}")
# print(f"Poids de la classe maligne : {malin_weight:.2f}")

class_weights = {0: 1, 1: 1}  # Augmenter le poids de la classe "malin"

@register_keras_serializable()
# Définition d'une métrique F1 personnalisée
class F1Score(Metric):
    def __init__(self, name="f1_score", **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

model.compile(
    optimizer='adam',  # Optimiseur Adam
    loss="binary_crossentropy",          # Fonction de perte pour la classification binaire
    metrics=["accuracy", Precision(name="precision"), Recall(name="recall"), F1Score()]  # Suivi de la précision
)

# Résumé du modèle
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 100, 100, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 4, 4, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 4, 4, 256)      │     2,949,376 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 2, 2, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 2, 2, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 1, 1, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,401,764 (35.86 MB)

 Trainable params: 9,359,741 (35.70 MB)

 Non-trainable params: 42,023 (164.16 KB)

In [27]:
# Visualisation de l'architecture du modèle
# plot_model(model, show_shapes=True, to_file="efficientnetb0_model.png")

## Entraînement du modèle


### Scheduler pour le Learning Rate
Le modèle est entraîné sur l’ensemble d’entraînement, avec un suivi sur l’ensemble de validation. Un learning rate dynamique est utilisé, évoluant selon les phases suivantes :
1. Augmentation progressive jusqu’à un seuil maximal.
2. Stabilisation au maximum pendant une durée définie.
3. Réduction exponentielle jusqu’à un seuil minimal.

Ce mécanisme permet une convergence optimale et une meilleure généralisation.

In [28]:
def learning_rate_function(epoch):
    LR_START = 0.000005 # Taux d'apprentissage initial
    LR_MAX = 0.000005 * strategy.num_replicas_in_sync # Taux d'apprentissage maximal
    LR_MIN = 0.000001 # Taux d'apprentissage minimal
    LR_RAMPUP_EPOCHS = 3 # nombre d'époques pendant lesquelles le taux d'apprentissage augmente linéairement.
    LR_SUSTAIN_EPOCHS = 0 # nombre d'époques où le taux reste maximal
    LR_EXP_DECAY = .8 # taux de décroissance exponentielle du taux d'apprentissage après les périodes de "ramp-up" et de "soutien"
    

    # Augmentation (pour les premières LR_RAMPUP_EPOCHS époques) : le taux d'apprentissage commence à LR_START et monte linéairement jusqu'à LR_MAX
    if epoch < LR_RAMPUP_EPOCHS: 
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START

    # Soutien (pour les LR_SUSTAIN_EPOCHS suivantes) : le taux d'apprentissage reste constant à LR_MAX
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    # Décroissance (pour les époques restantes) : le taux d'apprentissage diminue exponentiellement
    else:
        lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN
    return lr

lr_schedule = tf.keras.callbacks.LearningRateScheduler(learning_rate_function, verbose=1)

In [ ]:
# model = load_model(
#     "models/fine-tunning_40epochs.keras",
#     custom_objects={"F1Score": F1Score}
# )

## Enregistrement des logs avec TensorBoard et entraînement du modèle
Le modèle est entraîné tout en enregistrant les métriques dans TensorBoard pour un suivi en temps réel. On note les particularités suivantes:
- pour **TensorBoard** : Les logs sont enregistrés dans `logs/fit/model_name`, avec les histogrammes des poids activés (`histogram_freq=1`),
- pour l'**entraînement** : Le modèle utilise les ensembles d’entraînement et de validation, avec les callbacks suivants :
  - `lr_schedule` pour ajuster dynamiquement le learning rate.
  - `tensorboard_callback` pour visualiser les métriques dans TensorBoard.

In [ ]:
log_dir = os.path.join("logs", "fit", "model_name")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

history = model.fit(training_dataset, steps_per_epoch=steps_per_epochs, epochs=EPOCHS,
                    validation_data=validation_dataset, validation_steps=validation_steps,callbacks=[lr_schedule,tensorboard_callback],class_weight=class_weights,)

# tensorboard --logdir=logs/fit pour lancer TensorBoard


Epoch 1: LearningRateScheduler setting learning rate to 5e-06.
Epoch 1/40
 69/131 ━━━━━━━━━━━━━━━━━━━━ 29s 472ms/step - accuracy: 0.5175 - f1_score: 0.2882 - loss: 0.7005 - precision: 0.2194 - recall: 0.4365

In [ ]:
import json

# Sauvegarder les données d'history
with open('history.json', 'w') as f:
    json.dump(history.history, f)

## Évaluation et analyse des performances

### Affichage des courbes d'entraînement et de validation
Cette section génère et affiche les courbes d'évolution des métriques (`accuracy` et `loss`) au fil des époques pour évaluer les performances du modèle. Cela nous permet d’analyser :
- La convergence du modèle sur les données d'entraînement.
- La généralisation sur les données de validation

In [30]:
# Plot accuracy
plt.figure(figsize=(8, 4))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

# Plot loss
plt.figure(figsize=(8, 4))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.grid(True)
plt.show()

NameError: name 'history' is not defined

<Figure size 800x400 with 0 Axes>

In [ ]:
# Plot Precision, Recall, and F1-Score
plt.figure(figsize=(8, 4))
if 'precision' in history.history and 'val_precision' in history.history:
    plt.plot(history.history['precision'], label='Train Precision')
    plt.plot(history.history['val_precision'], label='Validation Precision')

if 'recall' in history.history and 'val_recall' in history.history:
    plt.plot(history.history['recall'], label='Train Recall')
    plt.plot(history.history['val_recall'], label='Validation Recall')

if 'f1_score' in history.history and 'val_f1_score' in history.history:
    plt.plot(history.history['f1_score'], label='Train F1-Score')
    plt.plot(history.history['val_f1_score'], label='Validation F1-Score')

plt.title('Model Metrics (Precision, Recall, F1-Score)')
plt.xlabel('Epoch')
plt.ylabel('Score')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

## Prédiction sur de nouvelles données

Cette section utilise le modèle entraîné pour prédire la classe (`benign` ou `malignant`) d'une image donnée. L'image est prétraitée pour correspondre au format attendu par le modèle, et la classe prédite est accompagnée d'une probabilité de confiance.

In [ ]:
# Prédire les résultats sur le dataset de test
predictions = model.predict(test2_dataset)

# Classification binaire (sigmoid), les prédictions sont des probabilités
predicted_probabilities = predictions.flatten()

# Convertir les probabilités en classes binaires (0 ou 1)
predicted_classes = (predicted_probabilities >= 0.5).astype(int)

# Calculer l'accuracy globale
true_labels = metadata_test2['label'].values  # Étiquettes réelles
accuracy = (predicted_classes == true_labels).mean()
print(f"Accuracy globale : {accuracy:.2%}")

# Calculer le nombre de bénins (classe 0) et malins (classe 1) correctement prédits
benign_correct = ((predicted_classes == 0) & (true_labels == 0)).sum()
malignant_correct = ((predicted_classes == 1) & (true_labels == 1)).sum()

# Nombre total de bénins et malins dans le dataset
benign_total = (true_labels == 0).sum()
malignant_total = (true_labels == 1).sum()

# Taux de réussite pour chaque classe
benign_accuracy = benign_correct / benign_total if benign_total > 0 else 0
malignant_accuracy = malignant_correct / malignant_total if malignant_total > 0 else 0

#Calculer les faux positifs (prédits comme 1 mais étiquetés 0) et les faux négatifs (prédits comme 0 mais étiquetés 1)
false_positives = ((predicted_classes == 1) & (true_labels == 0)).sum()
false_negatives = ((predicted_classes == 0) & (true_labels == 1)).sum()

print(f"Nombre total de cas bénins : {benign_total}, Prédits correctement : {benign_correct}, Taux de réussite : {benign_accuracy:.2%}")
print(f"Nombre total de cas malins : {malignant_total}, Prédits correctement : {malignant_correct}, Taux de réussite : {malignant_accuracy:.2%}")


print(f"Faux positifs (prédits malins, mais bénins) : {false_positives}")
print(f"Faux négatifs (prédits bénins, mais malins) : {false_negatives}")

# Tracer un histogramme des probabilités pour toutes les prédictions
plt.figure(figsize=(8, 6))
plt.hist(predicted_probabilities, bins=20, color='lightcoral', edgecolor='black')
plt.title('Distribution des probabilités des prédictions (toutes les images)')
plt.xlabel('Probabilité prédite')
plt.ylabel('Nombre d\'exemples')
plt.grid(True)
plt.show()

# Nombre d'exemples prédits comme classe 1
num_class_1 = (predicted_probabilities >= 0.5).sum()
print(f"Nombre d'exemples prédits comme classe 1 : {num_class_1}")

In [ ]:
from sklearn.metrics import roc_curve, auc

# Courbe ROC
# Récupérer les scores de prédiction (probabilités pour la classe positive)
y_score = predictions.ravel()  # Pour binaire

# Extraire les labels vrais (en supposant que 'label' soit une colonne de test_metadata)
y_true = metadata_test2['label'].values  # C'est un tableau 1D des étiquettes de classe

print(y_score)
print(y_true)

# Calcul de la courbe ROC
fpr, tpr, _ = roc_curve(y_true, y_score)
roc_auc = auc(fpr, tpr)

# Tracer la courbe ROC
plt.figure(figsize=(8, 4))
plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Random Guess')
plt.title('Receiver Operating Characteristic (ROC)')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()